In [1]:
import os
import tensorflow as tf
import numpy as np
import cPickle
import math
from configuration import *
os.environ["THEANO_FLAGS"] = "device=cuda2"
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.15
#os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
from caption_gen.CaptionWraper import *

Using cuDNN version 6020 on context None
Mapped name None to device cuda2: Tesla M40 24GB (0000:08:00.0)


In [3]:
from skipthought import encoder_manager

In [4]:
from storyteller.inference_utils import vocabulary
from storyteller.inference_utils import story_generator
from storyteller import inference_wrapper

In [5]:
FLAGS = tf.flags.FLAGS

tf.flags.DEFINE_string("checkpoint_path", "../StylishParaGen/bard_v1/res_adv_4/model.ckpt-174168",
                       "Model checkpoint file or directory containing a "
                       "model checkpoint file.")
tf.flags.DEFINE_string("vocab_file", "/media/VSlab3/kuanchen_arxiv/vocab_60000_advent_allpad/vocab_shared.txt", "Text file containing the vocabulary.")
tf.flags.DEFINE_string("book_data_dir", "/media/VSlab3/kuanchen_arxiv/BookCorpus_passage/", "directory of book data")
tf.flags.DEFINE_string("book_category", "Adventure", "cateogry of book")
#tf.flags.DEFINE_string("book_ft", "/media/VSlab3/kuanchen_arxiv/fairy_tale/", "fairy tale corpus")

tf.logging.set_verbosity(tf.logging.INFO)

In [6]:
vse_config = vse_config()

In [7]:
wrapper = CapgenWrapper(vse_config)

Loading image-sentence embedding...
Loading and initializing ConvNet...
Loading parameters...
Loading captions...
Embedding captions...


In [9]:
captions = wrapper.get_caption(image_loc="/media/VSlab3/kuanchen_arxiv/NeuralStoryTeller/neural-storyteller/images/universiades_girl.jpeg",k=100)

/home/kuanchen/anaconda2/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [10]:
captions

['A beautiful young woman cutting a mans hair .',
 "The young woman is cutting the young man 's hair .",
 'Woman with brown hair cutting a mans hair .',
 'A girl smiling and holding a white remote near other smiling people .',
 'A young girl using a phone while getting her hair done',
 'A woman with red lipstick on her lips standing next to a man with white hair .',
 'A young girl and a woman are smiling',
 'A woman is making a silly face while a lady looks on .',
 'a man with a tie behind a woman holding up her hair',
 'A woman smiles as she shows off her large hair and lips .',
 "A woman holds her dancing companion 's tie to her face .",
 'A woman holding her hand to her face next to a man .',
 'a woman holding some scissors and a comb is cutting a mans hair',
 'A woman waving for the camera and smiling .',
 'A girl using brush and comb on a boys hair .',
 'A woman holding a white game controller next to a woman with perky breast .',
 'A close up shot of a young lady with blonde hair

In [11]:
encoder = encoder_manager.EncoderManager()

In [12]:
encoder.load_model(model_config=stv_config(),
                   vocabulary_file="/media/VSlab3/kuanchen_arxiv/skip_thoughts_uni_2017_02_02/vocab.txt",
                   embedding_matrix_file="/media/VSlab3/kuanchen_arxiv/skip_thoughts_uni_2017_02_02/embeddings.npy",
                   checkpoint_path="/media/VSlab3/kuanchen_arxiv/skip_thoughts_uni_2017_02_02/model.ckpt-501424")

INFO:tensorflow:Reading vocabulary from /media/VSlab3/kuanchen_arxiv/skip_thoughts_uni_2017_02_02/vocab.txt
INFO:tensorflow:Loaded vocabulary with 930914 words.
INFO:tensorflow:Loading embedding matrix from /media/VSlab3/kuanchen_arxiv/skip_thoughts_uni_2017_02_02/embeddings.npy
INFO:tensorflow:Loaded embedding matrix with shape (930914, 620)
INFO:tensorflow:Building model.
INFO:tensorflow:Loading model from checkpoint: /media/VSlab3/kuanchen_arxiv/skip_thoughts_uni_2017_02_02/model.ckpt-501424
INFO:tensorflow:Restoring parameters from /media/VSlab3/kuanchen_arxiv/skip_thoughts_uni_2017_02_02/model.ckpt-501424
INFO:tensorflow:Successfully loaded checkpoint: model.ckpt-501424


In [13]:
vocab = vocabulary.Vocabulary(vocab_file=FLAGS.vocab_file, start_word='<sos>', end_word='<eos>',unk_word='<unk>')

INFO:tensorflow:Initializing vocabulary from file: /media/VSlab3/kuanchen_arxiv/vocab_60000_advent_allpad/vocab_shared.txt
INFO:tensorflow:Created vocabulary with 60003 words


In [14]:
storyteller_config = storyteller_config()
storyteller_config.vocab_size = len(vocab.vocab)

In [15]:
g = tf.Graph()
with g.as_default():
    model = inference_wrapper.InferenceWrapper()
    restore_fn = model.build_graph_from_config(storyteller_config,
                                               FLAGS.checkpoint_path)
g.finalize()

INFO:tensorflow:Building model.


In [16]:
description = np.mean(encoder.encode(captions),axis=0)

In [17]:
with open("./style_bias/bias_cap.pkl", 'r') as handle:
    bias_source = cPickle.load(handle)
with open("./style_bias/bias_advent_long_100.pkl", 'r') as handle:
    bias_target_long = cPickle.load(handle)
with open("./style_bias/bias_advent_short_100.pkl", 'r') as handle:
    bias_target_short = cPickle.load(handle)

In [18]:
description_style = description - bias_source + bias_target_long

In [19]:
sess = tf.Session(graph=g,config=config)

In [ ]:
np.expand_dims(description_style,0)

In [20]:
restore_fn(sess)

# Prepare the caption generator. Here we are implicitly using the default
# beam search parameters. See caption_generator.py for a description of the
# available beam search parameters.
generator = story_generator.StoryGenerator(model, vocab, max_caption_length=100, beam_size=)



stories = generator.beam_search(sess, np.expand_dims(description_style,0))
#captions = generator.beam_search(sess, stv_embed)
for i, story in enumerate(stories):
    # Ignore begin and end words.
    sentence = [vocab.id_to_word(w) for w in story.sentence[1:-1]]
    sentence = " ".join(sentence)
    print("  %d) %s (p=%f)" % (i, sentence, math.exp(story.logprob)))

INFO:tensorflow:Loading model from checkpoint: ../StylishParaGen/bard_v1/res_adv_4/model.ckpt-174168
INFO:tensorflow:Restoring parameters from ../StylishParaGen/bard_v1/res_adv_4/model.ckpt-174168
INFO:tensorflow:Successfully loaded checkpoint: model.ckpt-174168
  0) On the other side of the street , I saw a woman standing in the middle of the street . She was sitting on a bench in the middle of the road . (p=0.000000)
  1) On the other side of the street , I saw a woman standing in the middle of the street . She was standing in the middle of the street , looking out the window at the other end of the road . (p=0.000000)
  2) On the other side of the street , I saw a woman standing in the middle of the street . She was standing in the middle of the street , looking out the window at the other end of the street . A couple of men stood in front of her . (p=0.000000)
  3) On the other side of the street , I saw a woman standing in the middle of the street . She was standing in the middle 

In [ ]:
im